<a href="https://colab.research.google.com/github/FadelDia/streamit/blob/main/SmartFace_All_Phases(1st_%2C_2nd_%26_3rd).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PHASE 1 :
 DATA GATHERING & PREPROCESSING

Importons toutes nos données dans drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Importation des librairies nécessaires

In [ ]:
!pip install opencv-python-headless numpy


CRÉATION DES RÉPERTOIRES POUR CONSERVER LES IMAGES

In [ ]:
import os

# Chemins des répertoires d'images
directories = [
    r'C:\Users\fallo\Videos\DATA ANALYSIS WITH PYTHON\Pictures_4_Project\Donald_Trump',
    r'C:\Users\fallo\Videos\DATA ANALYSIS WITH PYTHON\Pictures_4_Project\Lewis_Hamilton',
    r'C:\Users\fallo\Videos\DATA ANALYSIS WITH PYTHON\Pictures_4_Project\Macky_Sall'
]

# Créer les répertoires si nécessaire
for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f'Création du répertoire : {directory}')
    else:
        print(f'Le répertoire existe déjà : {directory}')


Le répertoire existe déjà : C:\Users\fallo\Videos\DATA ANALYSIS WITH PYTHON\Pictures_4_Project\Donald_Trump
Le répertoire existe déjà : C:\Users\fallo\Videos\DATA ANALYSIS WITH PYTHON\Pictures_4_Project\Lewis_Hamilton
Le répertoire existe déjà : C:\Users\fallo\Videos\DATA ANALYSIS WITH PYTHON\Pictures_4_Project\Macky_Sall


INSTALLATON DE MCTNN

Choix du MTCNN au lieu du Haar Cascade
Le choix d'utiliser le MTCNN (Multi-task Cascaded Convolutional Networks) plutôt que le modèle Haar Cascade repose sur plusieurs raisons, notamment la précision, la robustesse et la facilité d'utilisation. Voici quelques points clés qui expliquent ce choix :

1. Précision et Robustesse
MTCNN :

Précision accrue : Le MTCNN utilise des réseaux de neurones convolutifs pour détecter les visages, ce qui le rend plus précis que le Haar Cascade. Il est capable de détecter des visages sous différents angles et dans des conditions d'éclairage variées.
Robustesse : Le MTCNN est moins sensible aux variations de l'éclairage, des expressions faciales et des angles de vue par rapport au Haar Cascade.
Haar Cascade :

Moins précis : Le modèle Haar Cascade repose sur des caractéristiques simples (haar-like features) et peut parfois manquer des visages ou générer de faux positifs, surtout dans des conditions d'éclairage et de pose variées.
Sensibilité aux conditions : Il est plus sensible aux variations d'éclairage et de pose, ce qui peut affecter ses performances.
2. Facilité d'Utilisation
MTCNN :

Moins de réglages nécessaires : Le MTCNN nécessite moins de réglages de paramètres (comme scaleFactor, minNeighbors) pour fonctionner correctement, ce qui simplifie son utilisation.
Prêt à l'emploi : Il est plus facile à utiliser et à intégrer dans les projets de reconnaissance faciale sans nécessiter de réglages fins pour différents scénarios.
Haar Cascade :

Réglages nécessaires : Le modèle Haar Cascade nécessite souvent des ajustements de paramètres pour obtenir de bonnes performances, ce qui peut être fastidieux et nécessiter des essais et erreurs.
3. Technologie Moderne
MTCNN :

Technologie plus récente : Le MTCNN est basé sur des réseaux de neurones convolutifs, ce qui en fait une technologie plus moderne et adaptée aux besoins actuels de la reconnaissance faciale.
Multi-tâches : En plus de la détection des visages, le MTCNN peut également détecter les points clés du visage (comme les yeux, le nez et la bouche), ce qui peut être utile pour des applications plus avancées.
Haar Cascade :

Technologie plus ancienne : Le modèle Haar Cascade est une technologie plus ancienne et, bien qu'il ait été très utile par le passé, il est souvent surpassé par des méthodes plus modernes comme le MTCNN.

In [ ]:
! pip install mtcnn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.2 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import numpy as np
from mtcnn import MTCNN

# Initialiser le détecteur MTCNN
detector = MTCNN()

def extract_face(image_path, output_path, size=(160, 160)):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Cannot read image {image_path}")
        return

    # Convertir l'image en RGB (MTCNN attend des images en RGB)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = detector.detect_faces(image_rgb)

    if len(results) == 0:
        print(f'No face found in {image_path}')
        return

    for result in results:
        x, y, width, height = result['box']
        x, y = max(0, x), max(0, y)
        face = image[y:y+height, x:x+width]
        face = cv2.resize(face, size)
        cv2.imwrite(output_path, face)
        break  # Assuming only one face per image

def load_faces(directory, output_directory):
    if not os.path.exists(directory):
        print(f"Directory {directory} does not exist.")
        return

    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for filename in os.listdir(directory):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(directory, filename)
            output_path = os.path.join(output_directory, filename)
            extract_face(image_path, output_path)

def load_dataset(base_directory, paths):
    for person, path in paths.items():
        for subset in ['train', 'test']:
            person_path = os.path.join(base_directory, subset, person)
            os.makedirs(person_path, exist_ok=True)
            load_faces(path, person_path)

def save_to_npz(base_directory, output_file):
    data = {}
    for subset in ['train', 'test']:
        subset_data = []
        subset_labels = []
        subset_dir = os.path.join(base_directory, subset)
        for person in os.listdir(subset_dir):
            person_dir = os.path.join(subset_dir, person)
            for filename in os.listdir(person_dir):
                if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                    image_path = os.path.join(person_dir, filename)
                    image = cv2.imread(image_path)
                    if image is not None:
                        subset_data.append(image)
                        subset_labels.append(person)
        data[f'{subset}_data'] = np.array(subset_data)
        data[f'{subset}_labels'] = np.array(subset_labels)
    np.savez_compressed(output_file, **data)

# Définir les chemins
base_directory = 'dataset'  # Répertoire de base où les données seront stockées
paths = {
    'Donald_Trump': r'C:\Users\fallo\Videos\DATA ANALYSIS WITH PYTHON\Pictures_4_Project\Donald_Trump',
    'Lewis_Hamilton': r'C:\Users\fallo\Videos\DATA ANALYSIS WITH PYTHON\Pictures_4_Project\Lewis_Hamilton',
    'Macky_Sall': r'C:\Users\fallo\Videos\DATA ANALYSIS WITH PYTHON\Pictures_4_Project\Macky_Sall'
}

# Exécuter le traitement
load_dataset(base_directory, paths)

# Enregistrer les données dans un fichier compressé .npz
output_file = 'faces_dataset.npz'
save_to_npz(base_directory, output_file)
print(f'Dataset saved to {output_file}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 850ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
1/1 ━━━━

Fonction save_to_npz :

Cette fonction parcourt les répertoires de formation et de test, lit les images traitées et les étiquettes (noms des personnes).
Elle stocke les images et les étiquettes dans des tableaux NumPy.
Les données sont ensuite enregistrées dans un fichier compressé .npz.
Exécution des étapes :

Nous chargeons les images, extrayons les visages, et les enregistrons dans les répertoires train et test.
Ensuite, nous utilisons save_to_npz pour compresser et enregistrer les données dans un fichier .npz.

Conclusion:

En utilisant MTCNN, vous bénéficiez d'une meilleure précision et robustesse pour la détection des visages, facilitant ainsi le traitement des images et améliorant la qualité des données pour l'étape de modélisation ultérieure.

PHASE 2 :
Features extraction with FaceNet/VGG16



Etape 1 :
Installation de keras-facenet

Installons le package keras-facenet en utilisant pip :

In [6]:
!pip install keras-facenet


  Preparing metadata (setup.py) ... done
  Created wheel for keras-facenet: filename=keras_facenet-0.3.2-py3-none-any.whl size=10367 sha256=5d9839c0734d62cac91c00655a180da6381ee4ff606a7b8cce83bd695f5bb631
  Stored in directory: /root/.cache/pip/wheels/1d/d8/a9/85cf04ea29321d2afcb82c0caaafdca9195385f9d68cbc7185
Successfully built keras-facenet


Etape 2 :
Création de la fonction get_embedding

Créons une fonction appelée get_embedding qui prend comme entrée le modèle d'embedding (FaceNet ou VGG16) et une image de visage. Cette fonction calcule une représentation incrustée de l'image du visage.

In [7]:
from keras_facenet import FaceNet
import numpy as np
import cv2

def get_embedding(model, face_image):
    # Redimensionner l'image à la taille requise par le modèle (160x160 pour FaceNet)
    face_image = cv2.resize(face_image, (160, 160))
    # Normaliser les valeurs des pixels
    face_image = face_image.astype('float32')
    mean, std = face_image.mean(), face_image.std()
    face_image = (face_image - mean) / std
    # Ajouter une dimension pour correspondre à l'entrée du modèle (1, 160, 160, 3)
    face_image = np.expand_dims(face_image, axis=0)
    # Obtenir l'incrustation
    embedding = model.embeddings(face_image)
    return embedding[0]


Etape 3 :  
Importation de VGG16 à partir du package Keras


In [8]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

# Charger le modèle VGG16 pré-entraîné
base_model = VGG16(include_top=False, input_shape=(224, 224, 3))
model = Model(inputs=base_model.inputs, outputs=base_model.layers[-1].output)

def get_embedding_vgg16(model, face_image):
    # Redimensionner l'image à la taille requise par le modèle (224x224 pour VGG16)
    face_image = cv2.resize(face_image, (224, 224))
    # Prétraiter l'image pour le modèle VGG16
    face_image = preprocess_input(face_image)
    # Ajouter une dimension pour correspondre à l'entrée du modèle (1, 224, 224, 3)
    face_image = np.expand_dims(face_image, axis=0)
    # Obtenir l'incrustation
    embedding = model.predict(face_image)
    return embedding[0]


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Étape 4 :
Appliquons la fonction et sauvegardons les résultats dans un fichier .npz


In [9]:
import os
import numpy as np
from keras_facenet import FaceNet

# Initialiser le modèle FaceNet
facenet_model = FaceNet()

# Répertoires des images traitées
directories = [
    r'/content/dataset/train/Donald_Trump',
    r'/content/dataset/train/Lewis_Hamilton',
    r'/content/dataset/train/Macky_Sall',
    r'/content/dataset/test/Donald_Trump',
    r'/content/dataset/test/Lewis_Hamilton',
    r'/content/dataset/test/Macky_Sall'
]

embeddings = []
labels = []

for directory in directories:
    for filename in os.listdir(directory):
        if filename.endswith('.jpg'):
            image_path = os.path.join(directory, filename)
            image = cv2.imread(image_path)
            embedding = get_embedding(facenet_model, image)
            embeddings.append(embedding)
            labels.append(directory.split(os.sep)[-2])  # Utiliser le nom du répertoire parent comme label

# Convertir les listes en tableaux numpy
embeddings = np.array(embeddings)
labels = np.array(labels)

# Sauvegarder les embeddings et les labels dans un fichier .npz
np.savez('face_embeddings.npz', embeddings=embeddings, labels=labels)


1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/

Résumé de la Deuxième Étape du Projet

Installation des dépendances :

Installation de la bibliothèque keras-facenet pour utiliser le modèle FaceNet.
Création de la fonction get_embedding :

Définition d'une fonction pour calculer la représentation intégrée (embedding) d'une image de visage en utilisant le modèle FaceNet.
Chargement des modèles pré-entraînés :

Utilisation du modèle FaceNet pour l'extraction des embeddings des visages.
Application de la fonction get_embedding :

Chargement des images de visages traitées (réparties en ensembles train et test).
Calcul des embeddings pour chaque image de visage.
Sauvegarde des résultats :

Stockage des embeddings et des labels associés dans un fichier .npz.
Cela permet de préparer les données nécessaires pour l'étape suivante du projet, en ayant les représentations intégrées des visages prêtes pour l'entraînement du modèle de reconnaissance faciale.

Phase 3: Classification des Embeddings et Déploiement

Cette phase consiste à classifier les représentations intégrées (embeddings) générées à l'étape précédente pour compléter le pipeline de reconnaissance d'images.

Instructions
Construction du Classifieur :

Construisons un classifieur SVM (Support Vector Machine) ou utiliser tout autre algorithme de classification jugé approprié pour la tâche.
Entraînement du Modèle :

Entraînons le modèle sur les images intégrées.
Évaluation du Modèle :

Évaluons le modèle à l'aide des métriques d'évaluation appropriées (précision, rappel, F1-score, etc.).
Essai d'Autres Classifieurs :

Testons et comparer d'autres classifieurs pour améliorer les performances du modèle.
Déploiement du Modèle avec Streamlit :


In [10]:
! pip install streamlit scikit-learn joblib numpy
! pip install mtcnn keras-facenet opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [11]:
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report


In [12]:
data = np.load('face_embeddings.npz')
train_embeddings = data['train_embeddings']
train_labels = data['train_labels']
test_embeddings = data['test_embeddings']
test_labels = data['test_labels']

label_encoder = LabelEncoder()
train_labels_enc = label_encoder.fit_transform(train_labels)
test_labels_enc = label_encoder.transform(test_labels)

KeyError: 'train_embeddings is not a file in the archive'

In [13]:
! pip install streamlit tensorflow scikit-learn joblib numpy mtcnn keras-facenet opencv-python

import numpy as np
import cv2
import joblib
import streamlit as st
from PIL import Image
from mtcnn.mtcnn import MTCNN
from keras_facenet import FaceNet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Charger les embeddings et les labels
data = np.load('face_embeddings.npz')  # Replace 'face_embeddings.npz' with your actual file

# Use the correct keys to access the data
X = data['embeddings']
y = data['labels']

# Encoder les labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Construire un modèle de réseau de neurones simple
model = Sequential([
    Dense(128, activation='relu', input_shape=(512,)),  # Adjusted input_shape to match embeddings
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Évaluer le modèle
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

print("Accuracy:", np.mean(y_pred_labels == y_test_labels))
print(classification_report(y_test_labels, y_pred_labels, target_names=label_encoder.classes_))

# Sauvegarder le modèle
model.save('face_recognition_model.h5')

# Déploiement avec Streamlit
detector = MTCNN()
embedder = FaceNet()

def extract_face(image, required_size=(160, 160)):
    faces = detector.detect_faces(image)
    if len(faces) == 0:
        return None
    x1, y1, width, height = faces[0]['box']
    x2, y2 = x1 + width, y1 + height
    face = image[y1:y2, x1:x2]
    face = cv2.resize(face, required_size)
    return face

def get_embedding(face):
    face = np.expand_dims(face, axis=0)
    embedding = embedder.embeddings(face)
    return embedding[0]

def main():
    st.title("Reconnaissance Faciale")
    st.write("Téléchargez une image pour prédire la personne.")

    uploaded_file = st.file_uploader("Choisissez une image...", type=["jpg", "jpeg", "png"])
    if uploaded_file is not None:
        image = Image.open(uploaded_file)
        image = np.array(image)

        face = extract_face(image)
        if face is not None:
            embedding = get_embedding(face)
            embedding = np.expand_dims(embedding, axis=0)
            prediction = model.predict(embedding)
            predicted_class = label_encoder.inverse_transform([np.argmax(prediction)])[0]
            st.write(f"Prédiction : {predicted_class}")
        else:
            st.write("Aucun visage détecté dans l'image téléchargée.")

if __name__ == '__main__':
    main()

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.4065 - loss: 0.7031 - val_accuracy: 0.4792 - val_loss: 0.6932
Epoch 2/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4424 - loss: 0.6940 - val_accuracy: 0.4792 - val_loss: 0.6970
Epoch 3/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4637 - loss: 0.6987 - val_accuracy: 0.4792 - val_loss: 0.6942
Epoch 4/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4874 - loss: 0.6970 - val_accuracy: 0.4792 - val_loss: 0.6940
Epoch 5/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4702 - loss: 0.6961 - val_accuracy: 0.4792 - val_loss: 0.6937
Epoch 6/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4951 - loss: 0.6926 - val_accuracy: 0.4792 - val_loss: 0.6937
Epoch 7/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4784 - loss: 0.6948 - val_accuracy: 0.4792 - val_loss: 0.6946
Epoch 8/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4649 - loss: 0.6984 - val_accuracy: 0.4792 - val_loss: 0.6935
Epoch 9/20
6/6 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.4791666666666667
              precision    recall  f1-score   support

        test       0.48      1.00      0.65        23
       train       0.00      0.00      0.00        25

    accuracy                           0.48        48
   macro avg       0.24      0.50      0.32        48
weighted avg       0.23      0.48      0.31        48



2024-08-09 18:31:30.405 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
